- title: "Differential Protein Expression"
- author: "Margaret Paiva"
- author: "Gilad Silberberg"
- date: "2021/12/21"
- output: R notebook

In [14]:
# cran packages
x <- c('dplyr',
       'data.table',
       'tidyr',   
       'tidyverse', 
       'ggplot2', 
       'viridis')
# bioconductor packages
y <- c('DEP', 
       'SummarizedExperiment')

In [12]:
# # install bioconductor packages
# if (!requireNamespace("BiocManager", quietly=TRUE))
#     install.packages("BiocManager")
# BiocManager::install(y)

In [14]:
# # install cran packages
# for (pkg in x) {
#     if (!pkg %in% rownames(installed.packages())) {install.packages(pkg)}
# }

In [17]:
# load packages
load_lib <- function(x) {
    suppressPackageStartupMessages(library(x, character.only = TRUE))
}
invisible(lapply(c(x, y), load_lib))

## Data

In [9]:
obj <- readRDS('./data/AMLproteomics_imp_norm_se_list2021-04-06.rds')
obj <- obj$conservative

In [10]:
obj

class: SummarizedExperiment 
dim: 2885 46 
metadata(0):
assays(1): ''
rownames(2885): A2M A8MWD9 ... ZW10 ZYX
rowData names(6): hgnc_symbol ProteinGroupId ... imputed num_NAs
colnames(46): CTG_2226 CTG_2227 ... CTG_3679 CTG_3680
colData names(26): ID condition ... White.blood.cell.count..x109.L.
  label

In [26]:
head(as.data.frame(rowData(obj)), 2)

,hgnc_symbol,ProteinGroupId,ID,name,imputed,num_NAs
,<chr>,<chr>,<chr>,<chr>,<lgl>,<dbl>
A2M,A2M,P01023,P01023,A2M,FALSE,0
A8MWD9,NA,A8MWD9,A8MWD9,A8MWD9,FALSE,0


In [24]:
head(as.data.frame(colData(obj)), 2)

,ID,condition,replicate,model_id,Age,BCR.ABL.status,CD34.status,Cytogenetics,Diagnosis,Ethnicity,...,Model,Model.status,NPM.status,Platelet.count..x103.ml.,Sample.source,Treatment.history,Tumor.type,TumorGraft.ID,White.blood.cell.count..x109.L.,label
,<chr>,<chr>,<dbl>,<fct>,<dbl>,<fct>,<fct>,<fct>,<fct>,<chr>,...,<fct>,<fct>,<chr>,<fct>,<fct>,<chr>,<fct>,<fct>,<dbl>,<chr>
CTG_2226,CTG_2226,CTG,2226,2226,52,NA,CD34-,"46, XY, ADD(6)(P21), DEL(8)(P21), ADD(12)(Q24.1) [13]/46, XY, DEL(1)(Q32), DEL(7)(Q22Q32), DER(6;12)(Q10;P10), ADD(22)(Q11.2), +MAR [3]/45, XY, T(1;2)(P22;Q11.2), -21 [1]/46, XY [3]",REFRACTORY,CAUCASIAN,...,CTG-2226,ESTABLISHED,WILD TYPE,23,LEUKAPHERESIS,PRETREATED,ACUTE MYELOID LEUKEMIA (AML),NA,113,CTG_2226
CTG_2227,CTG_2227,CTG,2227,2227,59,NA,CD34+,NA,RELAPSED,CAUCASIAN,...,CTG-2227,ESTABLISHED,MUTANT,51,LEUKAPHERESIS,PRETREATED,ACUTE MYELOID LEUKEMIA (AML),NA,288,CTG_2227


In [25]:
unique(colData(obj)$Treatment.history)

[1] "PRETREATED" "NAIVE"      NA

In [16]:
data <- UbiLength
# We filter for contaminant proteins and decoy database hits, 
# which are indicated by "+" in the columns "Potential.contaminants" and "Reverse", respectively. 
data <- filter(data, Reverse != "+", Potential.contaminant != "+")
head(data, 2)

,Protein.IDs,Majority.protein.IDs,Protein.names,Gene.names,Fasta.headers,Peptides,Razor...unique.peptides,Unique.peptides,LFQ.intensity.Ubi4_1,LFQ.intensity.Ubi4_2,...,LFQ.intensity.Ubi6_3,LFQ.intensity.Ctrl_1,LFQ.intensity.Ctrl_2,LFQ.intensity.Ctrl_3,LFQ.intensity.Ubi1_1,LFQ.intensity.Ubi1_2,LFQ.intensity.Ubi1_3,Only.identified.by.site,Reverse,Potential.contaminant
,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<int>,<dbl>,<dbl>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>
1,A0AV96-2;B7Z8Z7;A0AV96;D6R9D6;D6RBS9;D6REZ6;D6R9M7;D6RCT1;D6RBP6;D6RA49;D6RFL5;Q5T0W7;Q8TBY0-2;Q8TBY0-3;Q9NQ94-5;Q8TBY0;Q9NQ94-6;Q9NQ94-3;Q9NQ94-2;F8W9F8;Q9NQ94-4;Q9NQ94,A0AV96-2;B7Z8Z7;A0AV96;D6R9D6;D6RBS9,RNA-binding protein 47,RBM47,>sp|A0AV96-2|RBM47_HUMAN Isoform 2 of RNA-binding protein 47 OS=Homo sapiens GN=RBM47;>tr|B7Z8Z7|B7Z8Z7_HUMAN RNA-binding protein 47 OS=Homo sapiens GN=RBM47 PE=2 SV=1;>sp|A0AV96|RBM47_HUMAN RNA-binding protein 47 OS=Homo sapiens GN=RBM47 PE=1 SV=2;>tr|D6R,7,7,7,35787000,24701000,...,32705000,25898000,31275000,23076000,27664000,32314000,31097000,,,
2,A0AVT1;A0AVT1-2;H0Y8S8;A0AVT1-4;A0AVT1-3,A0AVT1;A0AVT1-2,Ubiquitin-like modifier-activating enzyme 6,UBA6,>sp|A0AVT1|UBA6_HUMAN Ubiquitin-like modifier-activating enzyme 6 OS=Homo sapiens GN=UBA6 PE=1 SV=1;>sp|A0AVT1-2|UBA6_HUMAN Isoform 2 of Ubiquitin-like modifier-activating enzyme 6 OS=Homo sapiens GN=UBA6,6,6,6,0,0,...,0,7260800,8612300,7816000,9907700,12387000,9023800,,,


In [17]:
# Make unique names using the annotation in the "Gene.names" column 
# as primary names and the annotation in "Protein.IDs" as name 
# for those that do not have an gene name.
data_unique <- make_unique(data, "Gene.names", "Protein.IDs", delim = ";")
head(data_unique, 2)

,Protein.IDs,Majority.protein.IDs,Protein.names,Gene.names,Fasta.headers,Peptides,Razor...unique.peptides,Unique.peptides,LFQ.intensity.Ubi4_1,LFQ.intensity.Ubi4_2,...,LFQ.intensity.Ctrl_2,LFQ.intensity.Ctrl_3,LFQ.intensity.Ubi1_1,LFQ.intensity.Ubi1_2,LFQ.intensity.Ubi1_3,Only.identified.by.site,Reverse,Potential.contaminant,name,ID
,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<int>,<dbl>,<dbl>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>
1,A0AV96-2;B7Z8Z7;A0AV96;D6R9D6;D6RBS9;D6REZ6;D6R9M7;D6RCT1;D6RBP6;D6RA49;D6RFL5;Q5T0W7;Q8TBY0-2;Q8TBY0-3;Q9NQ94-5;Q8TBY0;Q9NQ94-6;Q9NQ94-3;Q9NQ94-2;F8W9F8;Q9NQ94-4;Q9NQ94,A0AV96-2;B7Z8Z7;A0AV96;D6R9D6;D6RBS9,RNA-binding protein 47,RBM47,>sp|A0AV96-2|RBM47_HUMAN Isoform 2 of RNA-binding protein 47 OS=Homo sapiens GN=RBM47;>tr|B7Z8Z7|B7Z8Z7_HUMAN RNA-binding protein 47 OS=Homo sapiens GN=RBM47 PE=2 SV=1;>sp|A0AV96|RBM47_HUMAN RNA-binding protein 47 OS=Homo sapiens GN=RBM47 PE=1 SV=2;>tr|D6R,7,7,7,35787000,24701000,...,31275000,23076000,27664000,32314000,31097000,,,,RBM47,A0AV96-2
2,A0AVT1;A0AVT1-2;H0Y8S8;A0AVT1-4;A0AVT1-3,A0AVT1;A0AVT1-2,Ubiquitin-like modifier-activating enzyme 6,UBA6,>sp|A0AVT1|UBA6_HUMAN Ubiquitin-like modifier-activating enzyme 6 OS=Homo sapiens GN=UBA6 PE=1 SV=1;>sp|A0AVT1-2|UBA6_HUMAN Isoform 2 of Ubiquitin-like modifier-activating enzyme 6 OS=Homo sapiens GN=UBA6,6,6,6,0,0,...,8612300,7816000,9907700,12387000,9023800,,,,UBA6,A0AVT1
